In [2]:
#!pip install pycryptodome
#!pip install pycryptodomex

  Using cached pycryptodomex-3.9.7-cp37-cp37m-win_amd64.whl (14.1 MB)


In [3]:
from Crypto.PublicKey import RSA
from Cryptodome.Cipher import PKCS1_OAEP
from Cryptodome.Hash import SHA512 #,SHA256, SHA1
from Cryptodome.Signature import pss

import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

import base64

In [19]:

cred = credentials.Certificate("./covidhack-firebase-adminsdk-u4ged-d200bc2982.json")
firebase_admin.initialize_app(cred)

db = firestore.client()

ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.

In [37]:


def getdata(phnumber):
    
    
    privatekey = db.collection(u'Profile').document(u''+"+919958498955").get().to_dict()['Private']
    rsakey = RSA.importKey(base64.b64decode(privatekey))
    #rsakey =  PKCS1_OAEP.new(rsakey, hashAlgo=SHA256, mgfunc=lambda x,y: pss.MGF1(x,y, SHA256))
    rsakey = PKCS1_OAEP.new(rsakey, SHA512)
    
#     def decrypt(rsakey,loc)
#         global rsakey
#         b64_decoded_message = base64.b64decode(loc)
#         decrypted = rsakey.decrypt(b64_decoded_message)
#         return decrypted
    
    def decrypt_location(loc,rsakey):
#         global rsakey
        b64_decoded_message = base64.b64decode(loc)
        decrypted = rsakey.decrypt(b64_decoded_message)
        return decrypted
#         #temorary
#         #3decimal places taken
#         lat=round(locx[0],3)
#         long=round(locx[1],3)
#         return(str(lat)+','+str(long))
#         #temp ends
#         #decrypt and send
#         #return(list(map(float,loc.split(','))))
    def rev_cipher(num):
        #add code here
        return num

    users_ref = db.collection(u'Profile').document(u''+phnumber).collection('TimeStamps')
    docs = users_ref.stream()
    blth={}
    loc={}
    for doc in docs:
        bdic=doc.to_dict()
        time=bdic['TimeStamps']
        if(bdic['Activity']=="STILL"):
            loci=bdic['Location']
#             print(loci)
            loci=decrypt_location(loci,rsakey)
            if loci in loc:
                loc[loci] += 2
            else:
                loc[loci] = 2
        if 'BluetoothName' in bdic:
            for bl in bdic['BluetoothName']:
                bl1=rev_cipher(bl)
                if bl1 in blth:
                    blth[bl1] += 2
                else:
                    blth[bl1] = 2
    blth2={}
    for i in blth:
        #print(i)
        number=db.collection(u'Identify').document(u''+i).get().to_dict()['Number']
        blth2[number]=blth[i]
        
    return([blth2,loc])            
    

In [38]:
getdata(phnumber="+919958498955")

[{'+919831586263': 26}, {b'28.389,77.055': 12, b'28.389,77.056': 16}]